Fetching supplier data

Working with supplier images

In [ ]:
#!/usr/bin/env python3
from PIL import Image
import os, sys

path = "supplier-data/images/"
pictures = os.listdir(path)

for pic in pictures:
  if 'tiff' in pic:
    #grab the picture name without the .tiff extension
    file_name = os.path.splitext(pic)[0]
    outfile = "supplier-data/images/" + file_name + ".jpeg"
    try:
      Image.open(path + pic).convert("RGB").resize((600,400)).save(outfile,"JPEG")
    except IOError:
      print("cannot convert", pic)

Uploading images to web server

In [ ]:
#!/usr/bin/env python3
import os, sys
import requests

url = "http://localhost/upload/"
path = "supplier-data/images/"

images = os.listdir(path)

for image in images:
  if image.endswith(".jpeg"):
    with open(path + image, 'rb') as opened:
      r = requests.post(url, files={'file': opened})

Uploading the descriptions

In [ ]:
#!/usr/bin/env python3
import os, sys
import json
import requests

path = "supplier-data/descriptions/"
url = "http://localhost/fruits/"

files = os.listdir(path)

for file in files:
  if file.endswith("txt"):
    with open(path + file, 'r') as f:
      #grab the file name, ex. 001, 002 to use for image file
      fruit_name = os.path.splitext(file)[0]
      data = f.read()
      data = data.split("\n")
      fruit_dic = {"name": data[0], "weight": int(data[1].strip(" lbs")), "description": data[2], "image_name": fruit_name + ".jpeg"}
      response = requests.post(url, json=fruit_dic)
      response.raise_for_status()
      print(response.request.url)
      print(response.status_code)

Generate a PDF report and send it through email

In [ ]:
#!/usr/bin/env python3
from reportlab.platypus import SimpleDocTemplate
from reportlab.platypus import Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

def generate_report(filename, title, info):
    styles = getSampleStyleSheet()
    report = SimpleDocTemplate(filename)
    report_title = Paragraph(title, styles["h1"])
    report_info = Paragraph(info, styles["BodyText"])
    empty_line = Spacer(1,20)

    report.build([report_title, empty_line, report_info, empty_line])

In [ ]:
#!/usr/bin/env python3

import os, datetime
import reports
import emails

#get the current time in GMT
current_date = datetime.datetime.now().strftime('%Y-%m-%d')

def generate_pdf(path):
  pdf = ""
  files = os.listdir(path)
  for file in files:
    if file.endswith(".txt"):
      with open(path + file, 'r') as f:
        inline = f.readlines()
        name = inline[0].strip()
        weight = inline[1].strip()
        pdf += "name: " + name + "<br/>" + "weight: " + weight + "<br/><br/>"
  return pdf

if __name__ == "__main__":
  path = "supplier-data/descriptions/"
  title = "Process Updated on " + current_date 
  #generate the package for pdf body
  package = generate_pdf(path)
  reports.generate_report("/tmp/processed.pdf", title, package)

  #generate email information
  sender = "automation@example.com"
  receiver = "{}@example.com".format(os.environ["USER"])
  subject = "Upload Completed - Online Fruit Store"
  body = "All fruits are uploaded to our website successfully. A detailed list is attached to this email."
  attachment = "/tmp/processed.pdf"
  
  #generate email for the online fruit store report and pdf attachment
  message = emails.generate_email(sender, receiver, subject, body, attachment)
  emails.send_email(message)



In [ ]:
#!/usr/bin/env python3

import email.message
import mimetypes
import os.path
import smtplib

def generate_email(sender, recipient, subject, body, attachment_path = None):
  """Generate email, default is with no attachment"""
  # Basic Email formatting
  message = email.message.EmailMessage()
  message['Subject'] = subject
  message['From'] = sender
  message['To'] = recipient
  message.set_content(body)

  if attachment_path != None:
    attachment_name = os.path.basename(attachment_path)
    mime_type, _ = mimetypes.guess_type(attachment_path)
    mime_type, mime_subtype = mime_type.split("/", 1)
    with open(attachment_path, 'rb') as fp:
      message.add_attachment(fp.read(),
                            maintype=mime_type,
                            subtype=mime_subtype,
                            filename=attachment_name)
  return message

def send_email(package):
  """Sends the email package to the configured SMTP server."""
  mail_server = smtplib.SMTP('localhost')
  mail_server.send_message(package)
  mail_server.quit()

Health check

In [ ]:
#!/usr/bin/env python3

import psutil, shutil
import socket
import emails
import os, sys

#Warning if CPU usage is over 80%
def cpu_check():
  cpu_usage = psutil.cpu_percent(1) 
  return not cpu_usage > 80

#Report an error if available disk space is lower than 20%
def disc_space_check():  
  disk_usage = shutil.disk_usage("/")
  disk_total = disk_usage.total
  disk_free = disk_usage.used
  threshold = disk_free / disk_total * 100
  return threshold > 20

#Report an error if available memory is less than 500MB
def available_memory_check():
  available = psutil.virtual_memory().available
  available_in_MB = available / 1024 ** 2 #convert to MB
  return available_in_MB > 500

#Report an error if the hostname "localhost" cannot be resolved to "127.0.0.1"
def hostname_check():
  local_host_ip = socket.gethostbyname('localhost')
  return local_host_ip == "127.0.0.1"

def email_warning(error):
  sender = "automation@example.com"
  receiver = "{}@example.com".format(os.environ["USER"])
  subject = error
  body = "Please check your system and resolve the issue as soon as possible."
  message = emails.generate_email(sender, receiver, subject, body)
  emails.send_email(message)

if not cpu_check():
  subject = "Error - CPU usage is over 80%"
  email_warning(subject)

if not disc_space_check():
  subject = "Error - Available disk space is less than 20%"
  email_warning(subject)

if not available_memory_check():
  subject = "Error - Available memory is less than 500MB"
  email_warning(subject)

if not hostname_check():
  subject = "Error - localhost cannot be resolved to 127.0.0.1"
  email_warning(subject)